In [1]:
import tkinter as tk
from tkinter import ttk, filedialog, messagebox
from PIL import Image, ImageTk
import os
import numpy as np
from io import BytesIO
import random
import cv2
import sys
import os

# Функция для определения пути к ресурсам в EXE и в обычном режиме
def resource_path(relative_path):
    """Get absolute path to resource, works for dev and for PyInstaller"""
    try:
        # PyInstaller создает временную папку и хранит путь в _MEIPASS
        base_path = sys._MEIPASS
    except Exception:
        base_path = os.path.abspath(".")
    return os.path.join(base_path, relative_path)

class ComedyImageManipulator:
    def __init__(self, root):
        self.root = root
        self.root.title("Comedy Image Manipulator")
        self.root.geometry("1200x800")
        self.root.configure(bg="#2c3e50")
        
        # Языковые настройки
        self.language = "English"
        self.translations = {
            "English": {
                "title": "🎭 Comedy Image Manipulator",
                "load_image": "Load Image",
                "save_image": "Save Result",
                "compress_tab": "JPEG Compression",
                "upscale_tab": "Upscaling",
                "original": "Original Image",
                "result": "Result",
                "placeholder": "Load an image to begin",
                "result_placeholder": "Apply an operation to see result",
                "status_ready": "Ready to work",
                "compress_quality": "Compression Quality:",
                "compress_passes": "Compression Passes:",
                "apply_compress": "Apply Compression",
                "compress_desc": "Creates 'comedy' JPEG compression artifacts:",
                "upscale_scale": "Upscale Factor:",
                "upscale_noise": "Noise Level:",
                "color_shift": "Apply Color Distortions",
                "apply_upscale": "Apply Upscaling",
                "upscale_desc": "Creates 'comedy' upscaling effect:",
                "status_loading": "Loading: ",
                "status_compressing": "Applying compression...",
                "status_upscaling": "Applying upscaling...",
                "status_saving": "Saving: ",
                "error_load": "Error loading image: ",
                "error_compress": "Compression error: ",
                "error_upscale": "Upscaling error: ",
                "error_save": "Save error: ",
                "warning_no_image": "Please load an image first",
                "warning_no_result": "No result to save",
                "success_save": "Image saved successfully:\n"
            },
            "Russian": {
                "title": "🎭 Редактор Изображений",
                "load_image": "Загрузить изображение",
                "save_image": "Сохранить результат",
                "compress_tab": "JPEG Сжатие",
                "upscale_tab": "Увеличение",
                "original": "Оригинальное изображение",
                "result": "Результат",
                "placeholder": "Загрузите изображение",
                "result_placeholder": "Примените операцию для просмотра результата",
                "status_ready": "Готов к работе",
                "compress_quality": "Качество сжатия:",
                "compress_passes": "Количество проходов:",
                "apply_compress": "Применить сжатие",
                "compress_desc": "Создает 'комедийные' артефакты сжатия JPEG:",
                "upscale_scale": "Масштаб увеличения:",
                "upscale_noise": "Уровень шума:",
                "color_shift": "Применять цветовые искажения",
                "apply_upscale": "Применить увеличение",
                "upscale_desc": "Создает 'комедийный' эффект увеличения:",
                "status_loading": "Загружено: ",
                "status_compressing": "Применение сжатия...",
                "status_upscaling": "Применение увеличения...",
                "status_saving": "Сохранение: ",
                "error_load": "Ошибка загрузки изображения: ",
                "error_compress": "Ошибка сжатия: ",
                "error_upscale": "Ошибка увеличения: ",
                "error_save": "Ошибка сохранения: ",
                "warning_no_image": "Пожалуйста, загрузите изображение",
                "warning_no_result": "Нет результата для сохранения",
                "success_save": "Изображение успешно сохранено:\n"
            }
        }
        
        # Настройка стилей
        self.style = ttk.Style()
        self.style.theme_use('clam')
        self.style.configure('TFrame', background='#2c3e50')
        self.style.configure('TLabel', background='#2c3e50', foreground='#ecf0f1', font=('Arial', 10))
        self.style.configure('TButton', font=('Arial', 10), background='#3498db')
        self.style.map('TButton', background=[('active', '#2980b9')])
        self.style.configure('Header.TLabel', font=('Arial', 14, 'bold'))
        self.style.configure('TNotebook', background='#2c3e50')
        self.style.configure('TNotebook.Tab', background='#34495e', foreground='#ecf0f1', font=('Arial', 10, 'bold'))
        self.style.map('TNotebook.Tab', background=[('selected', '#3498db')])
        
        # Переменные
        self.image_path = None
        self.original_img = None
        self.result_img = None
        
        # Создание интерфейса
        self.create_widgets()
    
    def tr(self, key):
        """Возвращает перевод для текущего языка"""
        return self.translations[self.language].get(key, key)
    
    def toggle_language(self):
        """Переключает язык интерфейса"""
        self.language = "Russian" if self.language == "English" else "English"
        self.update_ui_text()
    
    def update_ui_text(self):
        """Обновляет все тексты в интерфейсе"""
        # Заголовок
        self.title_label.config(text=self.tr("title"))
        self.load_btn.config(text=self.tr("load_image"))
        self.save_btn.config(text=self.tr("save_image"))
        self.lang_btn.config(text="RU" if self.language == "English" else "EN")
        
        # Вкладки
        self.notebook.tab(0, text=self.tr("compress_tab"))
        self.notebook.tab(1, text=self.tr("upscale_tab"))
        
        # Области изображений
        self.original_frame.config(text=self.tr("original"))
        self.result_frame.config(text=self.tr("result"))
        
        # Заглушки
        self.original_placeholder.config(text=self.tr("placeholder"))
        self.result_placeholder.config(text=self.tr("result_placeholder"))
        
        # Статус бар
        self.status_var.set(self.tr("status_ready"))
        
        # Вкладка сжатия
        self.compress_qlabel_title.config(text=self.tr("compress_quality"))
        self.compress_plabel_title.config(text=self.tr("compress_passes"))
        self.compress_btn.config(text=self.tr("apply_compress"))
        self.compress_desc.config(text=self.tr("compress_desc"))
        
        # Вкладка апскейла
        self.upscale_slabel_title.config(text=self.tr("upscale_scale"))
        self.upscale_nlabel_title.config(text=self.tr("upscale_noise"))
        self.color_shift_cb.config(text=self.tr("color_shift"))
        self.upscale_btn.config(text=self.tr("apply_upscale"))
        self.upscale_desc.config(text=self.tr("upscale_desc"))
    
    def create_widgets(self):
        # Основные фреймы
        header_frame = ttk.Frame(self.root)
        header_frame.pack(fill=tk.X, padx=10, pady=10)
        
        # Заголовок
        self.title_label = ttk.Label(header_frame, text=self.tr("title"), style='Header.TLabel')
        self.title_label.pack(side=tk.LEFT)
        
        # Кнопка переключения языка
        self.lang_btn = ttk.Button(header_frame, text="RU", width=3, command=self.toggle_language)
        self.lang_btn.pack(side=tk.RIGHT, padx=5)
        
        # Кнопка загрузки изображения
        self.load_btn = ttk.Button(header_frame, text=self.tr("load_image"), command=self.load_image)
        self.load_btn.pack(side=tk.RIGHT, padx=5)
        
        # Создание вкладок
        self.notebook = ttk.Notebook(self.root)
        self.notebook.pack(fill=tk.X, padx=10, pady=5)
        
        # Вкладка компрессии
        compress_frame = ttk.Frame(self.notebook)
        self.notebook.add(compress_frame, text=self.tr("compress_tab"))
        self.create_compress_tab(compress_frame)
        
        # Вкладка апскейла
        upscale_frame = ttk.Frame(self.notebook)
        self.notebook.add(upscale_frame, text=self.tr("upscale_tab"))
        self.create_upscale_tab(upscale_frame)
        
        # Области изображений
        image_frame = ttk.Frame(self.root)
        image_frame.pack(fill=tk.BOTH, expand=True, padx=10, pady=(0, 5))
        
        # Оригинальное изображение
        self.original_frame = ttk.LabelFrame(image_frame, text=self.tr("original"))
        self.original_frame.pack(side=tk.LEFT, fill=tk.BOTH, expand=True, padx=5, pady=5)
        
        # Результат
        self.result_frame = ttk.LabelFrame(image_frame, text=self.tr("result"))
        self.result_frame.pack(side=tk.LEFT, fill=tk.BOTH, expand=True, padx=5, pady=5)
        
        # Заглушки для изображений
        self.original_placeholder = ttk.Label(self.original_frame, text=self.tr("placeholder"), 
                                             font=('Arial', 12), anchor=tk.CENTER)
        self.original_placeholder.pack(fill=tk.BOTH, expand=True)
        
        self.result_placeholder = ttk.Label(self.result_frame, text=self.tr("result_placeholder"), 
                                          font=('Arial', 12), anchor=tk.CENTER)
        self.result_placeholder.pack(fill=tk.BOTH, expand=True)
        
        # Создаем фрейм для статус-бара
        status_frame = ttk.Frame(self.root)
        status_frame.pack(side=tk.BOTTOM, fill=tk.X)
        
        # Кнопка сохранения результата (теперь в статус-баре)
        self.save_btn = ttk.Button(status_frame, text=self.tr("save_image"), command=self.save_image)
        self.save_btn.pack(side=tk.RIGHT, padx=(0, 10), pady=2)
        
        # Статус бар
        self.status_var = tk.StringVar(value=self.tr("status_ready"))
        status_bar = ttk.Label(status_frame, textvariable=self.status_var, relief=tk.SUNKEN, anchor=tk.W)
        status_bar.pack(side=tk.LEFT, fill=tk.X, expand=True, padx=(10, 0))
    
    def create_compress_tab(self, parent):
        # Переменные для компрессии
        self.compress_quality = tk.IntVar(value=5)
        self.compress_passes = tk.IntVar(value=3)
        
        # Фрейм параметров
        param_frame = ttk.Frame(parent)
        param_frame.pack(fill=tk.X, padx=10, pady=10)
        
        # Качество сжатия
        self.compress_qlabel_title = ttk.Label(param_frame, text=self.tr("compress_quality"))
        self.compress_qlabel_title.grid(row=0, column=0, padx=5, pady=5, sticky=tk.W)
        
        quality_slider = ttk.Scale(param_frame, from_=1, to=95, variable=self.compress_quality, 
                                  command=lambda v: self.compress_qlabel.config(text=f"{int(float(v))}"))
        quality_slider.grid(row=0, column=1, padx=5, sticky=tk.EW)
        
        self.compress_qlabel = ttk.Label(param_frame, text="5", width=3)
        self.compress_qlabel.grid(row=0, column=2, padx=5)
        
        # Количество проходов
        self.compress_plabel_title = ttk.Label(param_frame, text=self.tr("compress_passes"))
        self.compress_plabel_title.grid(row=1, column=0, padx=5, pady=5, sticky=tk.W)
        
        passes_slider = ttk.Scale(param_frame, from_=1, to=20, variable=self.compress_passes, 
                                 command=lambda v: self.compress_plabel.config(text=f"{int(float(v))}"))
        passes_slider.grid(row=1, column=1, padx=5, sticky=tk.EW)
        
        self.compress_plabel = ttk.Label(param_frame, text="3", width=3)
        self.compress_plabel.grid(row=1, column=2, padx=5)
        
        # Кнопка сжатия
        self.compress_btn = ttk.Button(param_frame, text=self.tr("apply_compress"), command=self.apply_compression)
        self.compress_btn.grid(row=0, column=3, rowspan=2, padx=10, sticky=tk.NS)
        
        # Описание
        desc_frame = ttk.Frame(parent)
        desc_frame.pack(fill=tk.X, padx=10, pady=5)
        
        self.compress_desc = ttk.Label(desc_frame, text=self.tr("compress_desc"))
        self.compress_desc.pack(fill=tk.X)
        
        # Настройка колонок
        param_frame.columnconfigure(1, weight=1)
    
    def create_upscale_tab(self, parent):
        # Переменные для апскейла
        self.upscale_scale = tk.IntVar(value=4)
        self.upscale_noise = tk.IntVar(value=30)
        self.apply_color_shift = tk.BooleanVar(value=True)
        
        # Фрейм параметров
        param_frame = ttk.Frame(parent)
        param_frame.pack(fill=tk.X, padx=10, pady=10)
        
        # Коэффициент масштабирования
        self.upscale_slabel_title = ttk.Label(param_frame, text=self.tr("upscale_scale"))
        self.upscale_slabel_title.grid(row=0, column=0, padx=5, pady=5, sticky=tk.W)
        
        scale_slider = ttk.Scale(param_frame, from_=2, to=8, variable=self.upscale_scale, 
                                command=lambda v: self.upscale_slabel.config(text=f"{int(float(v))}x"))
        scale_slider.grid(row=0, column=1, padx=5, sticky=tk.EW)
        
        self.upscale_slabel = ttk.Label(param_frame, text="4x", width=5)
        self.upscale_slabel.grid(row=0, column=2, padx=5)
        
        # Уровень шума
        self.upscale_nlabel_title = ttk.Label(param_frame, text=self.tr("upscale_noise"))
        self.upscale_nlabel_title.grid(row=1, column=0, padx=5, pady=5, sticky=tk.W)
        
        noise_slider = ttk.Scale(param_frame, from_=0, to=100, variable=self.upscale_noise, 
                               command=lambda v: self.upscale_nlabel.config(text=f"{int(float(v))}%"))
        noise_slider.grid(row=1, column=1, padx=5, sticky=tk.EW)
        
        self.upscale_nlabel = ttk.Label(param_frame, text="30%", width=5)
        self.upscale_nlabel.grid(row=1, column=2, padx=5)
        
        # Чекбокс для цветовой коррекции
        self.color_shift_cb = ttk.Checkbutton(param_frame, text=self.tr("color_shift"), 
                                        variable=self.apply_color_shift)
        self.color_shift_cb.grid(row=2, column=0, columnspan=2, padx=5, pady=5, sticky=tk.W)
        
        # Кнопка апскейла
        self.upscale_btn = ttk.Button(param_frame, text=self.tr("apply_upscale"), command=self.apply_upscale)
        self.upscale_btn.grid(row=0, column=3, rowspan=3, padx=10, sticky=tk.NS)
        
        # Описание
        desc_frame = ttk.Frame(parent)
        desc_frame.pack(fill=tk.X, padx=10, pady=5)
        
        self.upscale_desc = ttk.Label(desc_frame, text=self.tr("upscale_desc"))
        self.upscale_desc.pack(fill=tk.X)
        
        # Настройка колонок
        param_frame.columnconfigure(1, weight=1)
    
    def load_image(self):
        file_path = filedialog.askopenfilename(
            filetypes=[("Images", "*.jpg *.jpeg *.png *.bmp *.tiff")]
        )
        
        if not file_path:
            return
            
        try:
            self.image_path = file_path
            self.original_img = Image.open(file_path)
            
            # Показываем оригинал
            self.show_image(self.original_img, self.original_frame, self.original_placeholder)
            
            # Сбрасываем результат
            self.result_img = None
            self.result_placeholder.pack(fill=tk.BOTH, expand=True)
            for widget in self.result_frame.winfo_children():
                if isinstance(widget, (tk.Canvas, ttk.Label)) and widget != self.result_placeholder:
                    widget.destroy()
            
            self.status_var.set(f"{self.tr('status_loading')}{os.path.basename(file_path)} ({self.original_img.width}x{self.original_img.height})")
        except Exception as e:
            messagebox.showerror("Error", f"{self.tr('error_load')}{str(e)}")
    
    def show_image(self, image, frame, placeholder):
        # Удаляем предыдущее изображение
        placeholder.pack_forget()
        for widget in frame.winfo_children():
            if isinstance(widget, (tk.Canvas, ttk.Label)) and widget != placeholder:
                widget.destroy()
        
        # Масштабируем изображение для отображения (увеличим размер просмотра)
        max_size = (600, 500)  # Увеличим размер просмотра
        img_width, img_height = image.size
        
        # Рассчитываем размеры для отображения
        ratio = min(max_size[0] / img_width, max_size[1] / img_height)
        display_size = (int(img_width * ratio), int(img_height * ratio))
        
        # Создаем изображение для Tkinter
        display_img = image.copy()
        display_img.thumbnail(display_size, Image.LANCZOS)
        tk_img = ImageTk.PhotoImage(display_img)
        
        # Создаем холст для изображения
        canvas = tk.Canvas(frame, bg="#34495e", highlightthickness=0)
        canvas.pack(fill=tk.BOTH, expand=True)
        canvas.create_image(0, 0, anchor=tk.NW, image=tk_img)
        canvas.image = tk_img  # Сохраняем ссылку
        
        # Добавляем подпись с информацией
        info_label = ttk.Label(frame, text=f"Size: {img_width}x{img_height}",
                             anchor=tk.CENTER)
        info_label.pack(fill=tk.X, pady=5)
    
    def apply_compression(self):
        if not self.image_path or not self.original_img:
            messagebox.showwarning("Warning", self.tr("warning_no_image"))
            return
            
        try:
            self.status_var.set(self.tr("status_compressing"))
            self.root.update()
            
            img = self.original_img.copy()
            quality = self.compress_quality.get()
            passes = self.compress_passes.get()
            
            # Применяем комедийное сжатие
            current_img = img.copy()
            if current_img.mode != 'RGB':
                current_img = current_img.convert('RGB')
            
            for _ in range(passes):
                buffer = BytesIO()
                current_img.save(buffer, format='JPEG', quality=quality, optimize=True)
                buffer.seek(0)
                current_img = Image.open(buffer)
            
            self.result_img = current_img
            self.show_image(current_img, self.result_frame, self.result_placeholder)
            
            # Показываем информацию о размере
            buffer = BytesIO()
            current_img.save(buffer, format='JPEG', quality=quality)
            file_size = buffer.tell() / 1024  # в КБ
            
            orig_size = os.path.getsize(self.image_path) / 1024
            compression_ratio = orig_size / (file_size + 0.0001)  # избегаем деления на ноль
            
            self.status_var.set(f"Compression complete! Size: {file_size:.1f} KB (Ratio: {compression_ratio:.1f}x)")
            
        except Exception as e:
            messagebox.showerror("Error", f"{self.tr('error_compress')}{str(e)}")
            self.status_var.set(self.tr("error_compress"))
    
    def apply_upscale(self):
        if not self.image_path or not self.original_img:
            messagebox.showwarning("Warning", self.tr("warning_no_image"))
            return
            
        try:
            self.status_var.set(self.tr("status_upscaling"))
            self.root.update()
            
            # Конвертируем в OpenCV формат
            img = np.array(self.original_img.convert('RGB'))
            img = img[:, :, ::-1].copy()  # Конвертация RGB в BGR
            
            scale = self.upscale_scale.get()
            noise_level = self.upscale_noise.get()
            
            # 1. Первоначальное увеличение
            upscaled = cv2.resize(img, (0, 0), fx=scale, fy=scale, 
                                 interpolation=cv2.INTER_NEAREST)
            
            # 2. Добавление пиксельных артефактов
            pixelated = cv2.resize(upscaled, (upscaled.shape[1]//2, upscaled.shape[0]//2), 
                                  interpolation=cv2.INTER_NEAREST)
            pixelated = cv2.resize(pixelated, (upscaled.shape[1], upscaled.shape[0]), 
                                  interpolation=cv2.INTER_NEAREST)
            
            # 3. Смешивание с оригинальным апскейлом
            result = cv2.addWeighted(upscaled, 0.5, pixelated, 0.5, 0)
            
            # 4. Цветовые искажения (только если включены)
            if self.apply_color_shift.get():
                b, g, r = cv2.split(result)
                
                # Случайные смещения цветовых каналов
                offset_x = random.randint(-5, 5)
                offset_y = random.randint(-5, 5)
                b = np.roll(b, (offset_x, offset_y), (1, 0))
                g = np.roll(g, (offset_x//2, offset_y//2), (1, 0))
                
                # Усиление цветов
                r = cv2.multiply(r, 1.2)
                g = cv2.multiply(g, 0.9)
                
                result = cv2.merge([b, g, r])
            
            # 5. Добавление шума (только если уровень шума > 0)
            if noise_level > 0:
                noise = np.random.randint(-noise_level, noise_level, result.shape, dtype=np.int16)
                result = cv2.add(result.astype(np.int16), noise)
                result = np.clip(result, 0, 255).astype(np.uint8)
            
            # 6. Финальное размытие
            result = cv2.GaussianBlur(result, (3, 3), 0)
            
            # Конвертация обратно в PIL Image
            result_rgb = cv2.cvtColor(result, cv2.COLOR_BGR2RGB)
            self.result_img = Image.fromarray(result_rgb)
            
            # Показываем результат
            self.show_image(self.result_img, self.result_frame, self.result_placeholder)
            
            # Статус
            orig_size = f"{self.original_img.width}x{self.original_img.height}"
            new_size = f"{result.shape[1]}x{result.shape[0]}"
            color_status = "with color distortions" if self.apply_color_shift.get() else "without color distortions"
            noise_status = f", noise: {noise_level}%" if noise_level > 0 else ""
            self.status_var.set(f"Upscaling complete! {orig_size} → {new_size} ({color_status}{noise_status})")
            
        except Exception as e:
            messagebox.showerror("Error", f"{self.tr('error_upscale')}{str(e)}")
            self.status_var.set(self.tr("error_upscale"))
    
    def save_image(self):
        if not self.result_img:
            messagebox.showwarning("Warning", self.tr("warning_no_result"))
            return
            
        file_path = filedialog.asksaveasfilename(
            defaultextension=".jpg",
            filetypes=[("JPEG", "*.jpg"), ("PNG", "*.png"), ("All Files", "*.*")]
        )
        
        if not file_path:
            return
            
        try:
            self.status_var.set(f"{self.tr('status_saving')}{os.path.basename(file_path)}")
            self.root.update()
            
            self.result_img.save(file_path)
            messagebox.showinfo("Success", f"{self.tr('success_save')}{file_path}")
            self.status_var.set(f"Saved: {os.path.basename(file_path)}")
        except Exception as e:
            messagebox.showerror("Error", f"{self.tr('error_save')}{str(e)}")

if __name__ == "__main__":
    root = tk.Tk()
    
    try:
        # Используем функцию resource_path для получения правильного пути
        icon_path = resource_path("app_icon.ico")
        root.iconbitmap(icon_path)
    except Exception as e:
        print(f"Не удалось загрузить иконку: {e}")
        # Продолжаем без иконки
        
    app = ComedyImageManipulator(root)
    root.mainloop()